<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Generate/ProtXLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Generate protein sequences using ProtXLNet pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [ ]:
import torch
from transformers import XLNetLMHeadModel, XLNetTokenizer,
import re
import os
import requests
import pandas as pd
from tqdm.auto import tqdm
from Bio import SeqIO

<b>2. Load the vocabulary and ProtXLNet Model<b>

In [25]:
tokenizer = XLNetTokenizer.from_pretrained("Rostlab/prot_xlnet", do_lower_case=False)

In [26]:
model = XLNetLMHeadModel.from_pretrained("Rostlab/prot_xlnet")

<b>3. Load the model into the GPU if avilabile and switch to inference mode<b>

In [27]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

In [ ]:
model = model.to(device)
model = model.eval()

<b>4. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (unk)<b>

Secuencia de ejemplo, esta secuencia se tomo de la lista de secuencias usadas para entrenar el modelo, se selecciono al azar buscando la etiqueta positiva para secuencias que atacan tumores.

In [ ]:
sequences_Example = 'AAVALLPAVLLALLAPQLGKKKHRRRPSKKKRHW' 

In [37]:
sequences_Example = re.sub(r"[UZOB]", "<unk>", sequences_Example) 

<b>5. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [38]:
ids = tokenizer.encode(sequences_Example, add_special_tokens=False)

In [39]:
input_ids = torch.tensor(ids).unsqueeze(0).to(device)

<b>6. Generate Protein Sequence<b>

In [81]:
max_length = 17 #longitud promedio de secuencias que atacan cancer 17.014
temperature = 3.0
k = 0
p = 1.0
repetition_penalty = 1.0
num_return_sequences = 1000

In [82]:
output_ids = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=k,
        top_p=p,
        repetition_penalty=repetition_penalty,
        do_sample=True,
        num_return_sequences=num_return_sequences,
    )

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [77]:
output_sequences = ["".join("".join(tokenizer.decode(output_id)).split()) for output_id in output_ids]

In [78]:
print('Generated Sequences\n')
for output_sequence in output_sequences:
  print(output_sequence)

Generated Sequences

XRDNLSSMHXPVPVPVP
XYEHAHFENKKAKEEMM
XTPAHTHTHTR<sep>DTAQV
XXKDKKIMYNHNWAHDH
XNNDHMDMVAHNADHMM
XHWVAPECWLAPHSLQT
XEAIMTRIPIYPIITIL
XYVYIEIYWCCWWPIWC
XKHLKTKHLKTTSVTRH
XMDGGFCLLQSTAINNN
